<a href="https://colab.research.google.com/github/JeanCarloTS/Machine-Learning-Projects/blob/main/Est%C3%A1gio/Capta%C3%A7%C3%A3o_dados_Twitter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Referência única**

https://towardsdatascience.com/twitter-data-collection-tutorial-using-python-3267d7cfa93e

# **Bibliotecas e instalações**

In [ ]:
# Importando todas as bibliotecas requeridas
import tweepy                   # Biblioteca para acesso ao Twitter API usando Python
from google.colab import drive  # Importando o Google Drive para usar nesse Colab notebook
import json                     # Biblioteca para trabalhar com arquivos json
import csv                      # Biblioteca para trabalhar com arquivos csv
from datetime import date       # Biblioteca para capturar dados de data/horário
from datetime import datetime
import time                     # Biblioteca para contabilizar o tempo de execução do código
import pandas as pd             # Biblioteca para manipulação e analise de dados
import numpy as np              # Biblioteca para manipulações matemáticas (arrays, matrizes e etc)


# **Driver**

In [ ]:
# Conectando o Google Drive a este Colab Notebook
from google.colab import drive
drive.mount('/content/drive')   

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Criando uma variável, a qual conterá o caminho onde serão localizados os dados
path = '/content/drive/MyDrive/Colab Notebooks/Estágio - IMT & NPL Brasil/Primeira Semana'

# **Chaves do Twitter como desenvolvedor**

In [ ]:
api_key = 'hi1jHROTuqYXaCnotHQxveKdD'

api_secret_key = 'JzkHwN5wwBDlVMTvmGJovciAwh1PNLRQr3gmZbWpjDb6rgjDv2'

access_token = '1396838828263059457-nM3a0PfGomJa2BYjz2cj7qJWS4bcvE'

access_token_secret = 'l9C224MHomQL0HT3QVnG3QmqJra4ANCgIYEGUkopi459A'

In [ ]:
# Conecta ao Twitter API usando as chaves e tokens
auth = tweepy.OAuthHandler(api_key, api_secret_key)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth)

# **Funções proprietárias**

In [ ]:
# Função que facilita o armazenamento de dados em um arquivo JSON no Google Drive
# file_name: Nome do arquivo localizado no Google Drive
# file_content: O conteúdo ou dado que deseja ser salvo no arquivo
def save_json(file_name, file_content):
  with open(path + file_name, 'w', encoding='utf-8') as f:
    json.dump(file_content, f, ensure_ascii=False, indent=4)

In [ ]:
# Função para tratar o tempo limite do twitter API
def limit_handled(cursor, list_name):
  while True:
    try:
      yield cursor.next()
    # Captura a excessão do tempo limite do Twitter API e espera por 15 minutos
    except tweepy.RateLimitError:
      print("\nData points listados = {}".format(len(list_name)))
      print('Alcançou o tempo limite do Twitter API.')
      for i in range(3, 0, -1):
        print("Esperando por {} minutos.".format(i * 5))
        time.sleep(5 * 60)
    # Captura qualquer outra excessão do Twitter API
    except tweepy.error.TweepError:
      print('\nExcessão no TweepError capturada' )

In [ ]:
# Função para pegar tweets de um usuário específico 
# NOTE: Esse metódo tem capacidade de pegar apenas os 3200 tweets mais recentes.
# Source: https://gist.github.com/yanofsky/5436496

def get_all_tweets(screen_name):

  # Inicia uma lista para guardar todos os Tweets
  alltweets = []

  # Faz um requerimento inicial dos tweets mais recentes 
  # (200 é o maximo permitido)
  new_tweets = api.user_timeline(screen_name = screen_name,count=200)

  # Salva os tweets mais recentes
  alltweets.extend(new_tweets)

  # Salva o ID do ultimo tweet, permitindo que o processo seja repetido
  oldest = alltweets[-1].id - 1

  # Enquanto houver tweets restantes, continue capturando todos  
  while len(new_tweets) > 0:
    print("getting tweets before %s" % (oldest))
    # Todos as proximas requisições usam o parametro max_id para evitar duplicatas
    new_tweets = api.user_timeline(screen_name = screen_name,count=200,max_id=oldest)
    # Salva os tweets mais recentes
    alltweets.extend(new_tweets)
    # Salva o ID do ultimo tweet, permitindo que o processo seja repetido
    oldest = alltweets[-1].id - 1
    print("...%s tweets downloaded so far" % (len(alltweets)))
    ### END OF WHILE LOOP ###

  # Transformar todos os Tweets capturados pela tweepy em um array 2D que vai gerar conteúdo pro arquivo csv
  outtweets = [[tweet.id_str, tweet.created_at, tweet.text, tweet.favorite_count,tweet.in_reply_to_screen_name, tweet.retweeted] for tweet in alltweets]
  # Cria o csv
  with open(path + "_" + '%s_tweets.csv' % screen_name, 'w') as f:
    writer = csv.writer(f)
    writer.writerow(["id","created_at","text","likes","in reply to","retweeted"])
    writer.writerows(outtweets)
  pass

In [ ]:
# Função para salvar os objetos de seguidores em um aquivo JSON.
def get_followers():
  
  # Cria uma lista para armazenar os dados de seguidores 
  followers_list = []
  # For-loop para interagir com o cursor da biblioteca tweepy
  cursor = tweepy.Cursor(api.followers, count=200).pages()
  for i, page in enumerate(limit_handled(cursor, followers_list)):  
    print("\r"+"Loading"+ i % 5 *".", end='')
    
    # Adiciona o ultimo conjunto de seguidores à lista
    followers_list += page
  
  # Extrai o conteúdo dos seguidores
  followers_list = [x._json for x in followers_list]
  # Salva os dados em um arquivo JSON
  save_json('followers_data.json', followers_list)

In [ ]:
# Função para salvar os objetos de amigos em um aquivo JSON.
def get_friends():
  
  # Cria uma lista para armazenar os dados de amigos 
  friends_list = []
  # For-loop para interagir com o cursor da biblioteca tweepy
  cursor = tweepy.Cursor(api.friends, count=200).pages()
  for i, page in enumerate(limit_handled(cursor, friends_list)):  
    print("\r"+"Loading"+ i % 5 *".", end='')
    
    # Adiciona o ultimo conjunto de amigos à lista
    friends_list += page
  
  # Extrai o conteúdo dos amigos
  friends_list = [x._json for x in friends_list]
  # Salva os dados em um arquivo JSON
  save_json('friends_data.json', friends_list)

In [ ]:
# Função para salvar numero de seguidos e seguidores em um arquivo JSON
def todays_stats(dict_name):
  # Pega a minha informação de usuário 
  info = api.me()
  # Conta o numero de seguidos e seguidores
  followers_cnt = info.followers_count  
  following_cnt = info.friends_count
  # Armazena a data de hoje
  today = date.today()
  d = today.strftime("%b %d, %Y")
  # Se não foram capturadas, salva o status do dia
  if d not in dict_name:
    dict_name[d] = {"followers":followers_cnt, "following":following_cnt}
    save_json("follower_history.json", dict_name)
  else:
    print('Today\'s stats already exist')

# **Uso das funções**

In [ ]:
get_all_tweets('jornaldobrasil')

getting tweets before 1396906575130996735
...399 tweets downloaded so far
getting tweets before 1393556913456091149
...598 tweets downloaded so far
getting tweets before 1390453532831191042
...798 tweets downloaded so far
getting tweets before 1387596687577845762
...998 tweets downloaded so far
getting tweets before 1383793846518116352
...1198 tweets downloaded so far
getting tweets before 1380923443231752206
...1397 tweets downloaded so far
getting tweets before 1378498319685652482
...1595 tweets downloaded so far
getting tweets before 1375910274838974466
...1795 tweets downloaded so far
getting tweets before 1373103846470344705
...1994 tweets downloaded so far
getting tweets before 1370174275030822915
...2194 tweets downloaded so far
getting tweets before 1368254377664933887
...2393 tweets downloaded so far
getting tweets before 1365650670133661699
...2592 tweets downloaded so far
getting tweets before 1363624031115767811
...2791 tweets downloaded so far
getting tweets before 1360373

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Estágio - IMT & NPL Brasil/Primeira Semana_jornaldobrasil_tweets.csv')
df

,id,created_at,text,likes,in reply to,retweeted
0,1399352005051793426,2021-05-31 13:08:11,"[DO LEITOR]\n""O governador de Pernambuco @Paul...",2,NaN,False
1,1399351075073822729,2021-05-31 13:04:29,RT @brasil247: Globalistas - Como a imprensa i...,0,NaN,False
2,1399350566644551687,2021-05-31 13:02:28,RT @RhangelRibeiro: Tá aí o Pastor Cláudio Dua...,0,NaN,False
3,1399350475963633664,2021-05-31 13:02:06,"RT @davidmirandario: Homens armados, usando as...",0,NaN,False
4,1399350402303381504,2021-05-31 13:01:49,RT @GeorgMarques: Globo e Estadão estão em cam...,0,NaN,False
...,...,...,...,...,...,...
3235,1351684782925737984,2021-01-20 00:15:39,"RT @cirogomes: Que a China, a Índia e os demai...",0,NaN,False
3236,1351684717268107267,2021-01-20 00:15:24,RT @AndreaMPacha: Como é possível que diante d...,0,NaN,False
3237,1351684659810349058,2021-01-20 00:15:10,RT @reinaldoazevedo: Argentina fechou acordo c...,0,NaN,False
3238,1351684609008955395,2021-01-20 00:14:58,RT @fabioluizfranca: Dilma caiu por pedalar. B...,0,NaN,False
